In [1]:
# !pip install jupyter-dash
# !pip install Flask --upgrade

from jupyter_dash import JupyterDash
from dash import html, dcc
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
# Date / Time 
import calendar
from datetime import datetime

# Country colors for plots theme
country_colors = {'Ireland': "green", 'Netherlands': "orange", 'New Zealand': "grey"}

exports = pd.read_csv('Data/projections_for_dash.csv')
exports['Month Date'] = exports['Month Date'].map(lambda x: datetime.strptime(x,'%Y-%m-%d'))

# Only yearly observations needed
exports = exports.groupby(['Country','Partner','Partner Code','Year']).sum().reset_index()

app = JupyterDash(__name__)

partner_dropdown = dcc.Dropdown(options=exports['Partner'].unique(),value='United States of America')
country_dropdown = dcc.Dropdown(options=exports['Country'].unique(),value='Ireland')

app.layout = html.Div(children=[
    # Line graph showing history and projections of the three expoorting countries
    html.Div([
        html.H1(children='Butter Exports Dashboard'),
        partner_dropdown,
        dcc.Graph(
            id='quantity-graph'
        ),
    ]),
    # Choropleth, by exporting country
    html.Div([
#         html.H1(children='Export partners by country'),
        country_dropdown,
        dcc.Graph(
            id='choropleth'
        ),
    ]),
])

@app.callback(
    Output(component_id='quantity-graph', component_property='figure'),
    Input(component_id=partner_dropdown, component_property='value')
)
def update_graph(selected_partner,):
    filtered_exports = exports[exports['Partner'] == selected_partner]
    line_fig = px.line(filtered_exports,
                       x='Year', y='Quantity (tonnes)',
                       color='Country',
                       color_discrete_map=country_colors,
                       title=f'Export Quantities (tonnes) to {selected_partner}. Select alternative trading partners above.')
    return line_fig

@app.callback(
    Output(component_id='choropleth', component_property='figure'),
    Input(component_id=country_dropdown, component_property='value')
)
def update_choropleth(selected_country,):
    filtered_by_country = exports[exports['Country'] == selected_country].sort_values(by='Year')
    choropleth_fig = px.choropleth(filtered_by_country,
                                locations="Partner Code",
                                color="Quantity (tonnes)",
                                hover_name="Partner",
                                animation_frame="Year",
                                color_continuous_scale='greens',
                                height=600,
                                title=f'Export destinations from {selected_country}. Select alternative exporting country above.')

    return choropleth_fig

app.run_server(mode='inline')